In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/biggest-brazilian-cities-by-population/Population_Brazilian_Cities_V2.csv
/kaggle/input/biggest-brazilian-cities-by-population/Population_Brazilian_Cities.csv
/kaggle/input/brazilian-holidays-national-state-city/HP Forecasting - Brazilian_Holidays.csv
/kaggle/input/brazilian-holidays-national-state-city/Brazilian_Holidays.csv
/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv
/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv
/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv
/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv


In [2]:
!pip install polars

In [3]:
import polars as pl
from datetime import datetime, timedelta
import numpy as np

In [4]:
# Import Datasets
customers_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv").collect()
sellers_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv").collect()
reviews_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv").collect()
ord_items_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv").collect()
products_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv").collect()
geos_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv").collect()
translation_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv").collect()
orders_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv").collect()
payments_df = pl.scan_csv("/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv").collect()

In [5]:
state_region = geos_df.lazy().select(
    [pl.col("geolocation_state"),
     pl.col("geolocation_city")
    ]
).unique().collect()

print(state_region)

shape: (8463, 2)
┌───────────────────┬───────────────────────┐
│ geolocation_state ┆ geolocation_city      │
│ ---               ┆ ---                   │
│ str               ┆ str                   │
╞═══════════════════╪═══════════════════════╡
│ SP                ┆ sao paulo             │
│ SP                ┆ são paulo             │
│ SP                ┆ sao bernardo do campo │
│ RN                ┆ são paulo             │
│ ...               ┆ ...                   │
│ RS                ┆ santa cecilia do sul  │
│ RS                ┆ ciríaco               │
│ RS                ┆ estação               │
│ RS                ┆ vila lângaro          │
└───────────────────┴───────────────────────┘


In [6]:
# Feature 1 -- Total Payment Value
pay_df = payments_df.lazy().select(pl.col(["order_id","payment_value"])).collect()

print(pay_df)

shape: (103886, 2)
┌──────────────────────────────────┬───────────────┐
│ order_id                         ┆ payment_value │
│ ---                              ┆ ---           │
│ str                              ┆ f64           │
╞══════════════════════════════════╪═══════════════╡
│ b81ef226f3fe1789b1e8b2acac839d17 ┆ 99.33         │
│ a9810da82917af2d9aefd1278f1dcfa0 ┆ 24.39         │
│ 25e8ea4e93396b6fa0d3dd708e76c1bd ┆ 65.71         │
│ ba78997921bbcdc1373bb41e913ab953 ┆ 107.78        │
│ ...                              ┆ ...           │
│ 7b905861d7c825891d6347454ea7863f ┆ 96.8          │
│ 32609bbb3dd69b3c066a6860554a77bf ┆ 47.77         │
│ b8b61059626efa996a60be9bb9320e10 ┆ 369.54        │
│ 28bbae6599b09d39ca406b747b6632b1 ┆ 191.58        │
└──────────────────────────────────┴───────────────┘


In [7]:
#Feature(s) 2A -- Num items per order

num_items_per_order_df = ord_items_df.lazy().select(
    [
        pl.col("order_id"),
        pl.col("order_item_id").unique().count().over("order_id").alias("num_items_per_order")
    
    ]).collect()


print(num_items_per_order_df.head(20))

shape: (20, 2)
┌──────────────────────────────────┬─────────────────────┐
│ order_id                         ┆ num_items_per_order │
│ ---                              ┆ ---                 │
│ str                              ┆ u32                 │
╞══════════════════════════════════╪═════════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                   │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                   │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                   │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                   │
│ ...                              ┆ ...                 │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                   │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                   │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                   │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                   │
└──────────────────────────────────┴─────────────────────┘


In [8]:
#Feature(s) 2A -- Num categories per order

num_cat_per_order_df = ord_items_df.lazy().join(products_df.lazy(), how="left", on="product_id").select(pl.col(["order_id", "order_item_id", "product_category_name"])).select(
    [
        pl.col("order_id"),
        pl.col("product_category_name").unique().count().over("order_id").alias("num_cat_per_order")
    
    ]).collect()

print(num_cat_per_order_df.head(20))


shape: (20, 2)
┌──────────────────────────────────┬───────────────────┐
│ order_id                         ┆ num_cat_per_order │
│ ---                              ┆ ---               │
│ str                              ┆ u32               │
╞══════════════════════════════════╪═══════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                 │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                 │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                 │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                 │
│ ...                              ┆ ...               │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                 │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                 │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                 │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                 │
└──────────────────────────────────┴───────────────────┘


In [9]:
# Feature 3A - Customer location
# orders_df [order_id, customer_id
# customers_df [customer_id, customer_city, customer_state]]


customer_locat_df = orders_df.lazy().select(pl.col(["order_id", "customer_id"])).join(customers_df.lazy().select(pl.col(["customer_id", "customer_city", "customer_state"])), how="left", on="customer_id").collect()

print(customer_locat_df.head(20))

shape: (20, 4)
┌───────────────────────────┬───────────────────────────┬─────────────────────────┬────────────────┐
│ order_id                  ┆ customer_id               ┆ customer_city           ┆ customer_state │
│ ---                       ┆ ---                       ┆ ---                     ┆ ---            │
│ str                       ┆ str                       ┆ str                     ┆ str            │
╞═══════════════════════════╪═══════════════════════════╪═════════════════════════╪════════════════╡
│ e481f51cbdc54678b7cc49136 ┆ 9ef432eb6251297304e76186b ┆ sao paulo               ┆ SP             │
│ f2d6af7                   ┆ 10a928d                   ┆                         ┆                │
│ 53cdb2fc8bc7dce0b6741e215 ┆ b0830fb4747a6c6d20dea0b8c ┆ barreiras               ┆ BA             │
│ 0273451                   ┆ 802d7ef                   ┆                         ┆                │
│ 47770eb9100c2d0c44946d9cf ┆ 41ce2a54c0b03bf3443c3d931 ┆ vianopolis        

In [10]:
# Feature - Number of sellers

#orders_df [order_id]
#ord_items_df [order_id, order_item_id, seller_id]
#sellers_df [seller_id,seller_city,seller_state]

# First get sellers per order_item
# Second get number of selers per order

num_sellers_per_order_df = ord_items_df.lazy().join(sellers_df.lazy(), how="left", on="seller_id").select([
    pl.col("order_id"),
    pl.col("seller_id").unique().count().over("order_id").alias("num_sellers_per_order")
]
).collect()

num_cities_per_order_df = ord_items_df.lazy().join(sellers_df.lazy(), how="left", on="seller_id").select([
    pl.col("order_id"),
    pl.col("seller_city").unique().count().over("order_id").alias("num_seller_cities_per_order")
]
).collect()

print(num_sellers_per_order_df.head(20))

print(num_cities_per_order_df.head(20))


shape: (20, 2)
┌──────────────────────────────────┬───────────────────────┐
│ order_id                         ┆ num_sellers_per_order │
│ ---                              ┆ ---                   │
│ str                              ┆ u32                   │
╞══════════════════════════════════╪═══════════════════════╡
│ 00010242fe8c5a6d1ba2dd792cb16214 ┆ 1                     │
│ 00018f77f2f0320c557190d7a144bdd3 ┆ 1                     │
│ 000229ec398224ef6ca0657da4fc703e ┆ 1                     │
│ 00024acbcdf0a6daa1e931b038114c75 ┆ 1                     │
│ ...                              ┆ ...                   │
│ 0009c9a17f916a706d71784483a5d643 ┆ 1                     │
│ 000aed2e25dbad2f9ddb70584c5a2ded ┆ 1                     │
│ 000c3e6612759851cc3cbb4b83257986 ┆ 1                     │
│ 000e562887b1f2006d75e0be9558292e ┆ 1                     │
└──────────────────────────────────┴───────────────────────┘
shape: (20, 2)
┌──────────────────────────────────┬───────────────────

In [11]:
# Feature: Was this order purchased on a holiday week for either the customer or the seller?
# order purchase timestamp for either customer or seller was within 7 days of a holiday
# for either seller or buyer

# First add two columns to holiday DF -- subtract 3 days prior & 3 days after

# First check: Are there rows where order purchase date-time stamp in-between dates? 

# If No: 
    # Then IsHolidayWeek = "False"
# if Yes: Return rows where order purchase month-day between start_month_day & end_month_day 
    # Check if Holiday_Type ="National" 
    # If Yes, IsHolidayWeek = "True"
#Else if State, Check States Match
    # If Yes, IsHolidayWeek = "True"
# Else if City, Check City Match
    # If Yes, IsHolidayWeek = "True"
# Else IsHolidayWeek = "True"

# Return IsHolidayWeek

In [12]:
# Added Additional Datasets

brazilian_holidays_df = pl.scan_csv("/kaggle/input/brazilian-holidays-national-state-city/HP Forecasting - Brazilian_Holidays.csv").collect()
brazilian_cities_population_df = pl.scan_csv("/kaggle/input/biggest-brazilian-cities-by-population/Population_Brazilian_Cities_V2.csv").collect()

In [13]:
print(brazilian_cities_population_df)

shape: (115, 5)
┌──────────────┬──────────────────┬─────────────────┬─────────────────┬─────────────────┐
│ State_Abbrev ┆ State            ┆ City_Lower      ┆ City            ┆ City_Population │
│ ---          ┆ ---              ┆ ---             ┆ ---             ┆ ---             │
│ str          ┆ str              ┆ str             ┆ str             ┆ str             │
╞══════════════╪══════════════════╪═════════════════╪═════════════════╪═════════════════╡
│ SP           ┆ São Paulo        ┆ são paulo       ┆ São Paulo       ┆ 12,033,866      │
│ RJ           ┆ Rio de Janeiro   ┆ rio de janeiro  ┆ Rio de Janeiro  ┆ 6,704,663       │
│ BA           ┆ Bahia            ┆ salvador        ┆ Salvador        ┆ 2,886,775       │
│ DF           ┆ Distrito Federal ┆ brasília        ┆ Brasília        ┆ 3,050,336       │
│ ...          ┆ ...              ┆ ...             ┆ ...             ┆ ...             │
│ RJ           ┆ Rio de Janeiro   ┆ itaboraí        ┆ Itaboraí        ┆ 228,996     

In [14]:
print(brazilian_holidays_df)

shape: (120, 6)
┌──────────┬───────┬──────────────┬──────────────┬────────────────────────┬────────────────────────┐
│ Type     ┆ State ┆ City_Lower   ┆ Holiday_Date ┆ English_Name           ┆ Portuguese_Name        │
│ ---      ┆ ---   ┆ ---          ┆ ---          ┆ ---                    ┆ ---                    │
│ str      ┆ str   ┆ str          ┆ str          ┆ str                    ┆ str                    │
╞══════════╪═══════╪══════════════╪══════════════╪════════════════════════╪════════════════════════╡
│ National ┆ All   ┆ all          ┆ 1/1/2017     ┆ New Year's Day         ┆ Ano Novo /             │
│          ┆       ┆              ┆              ┆                        ┆ Confraternização       │
│          ┆       ┆              ┆              ┆                        ┆ Univ...                │
│ National ┆ All   ┆ all          ┆ 4/21/2017    ┆ Tiradentes' Day        ┆ Dia de Tiradentes      │
│ National ┆ All   ┆ all          ┆ 5/1/2017     ┆ Labour Day             ┆

In [15]:
# Step 1: Get Customers by Order
get_customer_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "customer_id","order_purchase_timestamp"])).join(customers_df.lazy().select(pl.col(["customer_id", "customer_city", "customer_state"])), how="left", on="customer_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()

print(get_customer_regions_by_order.head(20))

shape: (20, 5)
┌────────────────────┬────────────────────┬───────────────────┬───────────────────┬────────────────┐
│ order_id           ┆ customer_id        ┆ order_purchase_ti ┆ customer_city     ┆ customer_state │
│ ---                ┆ ---                ┆ mestamp           ┆ ---               ┆ ---            │
│ str                ┆ str                ┆ ---               ┆ str               ┆ str            │
│                    ┆                    ┆ datetime[μs]      ┆                   ┆                │
╞════════════════════╪════════════════════╪═══════════════════╪═══════════════════╪════════════════╡
│ e481f51cbdc54678b7 ┆ 9ef432eb6251297304 ┆ 2017-10-02        ┆ sao paulo         ┆ SP             │
│ cc49136f2d6af7     ┆ e76186b10a928d     ┆ 00:00:00          ┆                   ┆                │
│ 53cdb2fc8bc7dce0b6 ┆ b0830fb4747a6c6d20 ┆ 2018-07-24        ┆ barreiras         ┆ BA             │
│ 741e2150273451     ┆ dea0b8c802d7ef     ┆ 00:00:00          ┆             

In [16]:
# Step 2: Get sellers By Order
get_sellers_regions_by_order = orders_df.lazy().select(pl.col(["order_id", "order_purchase_timestamp"])).join(ord_items_df.lazy().select(pl.col(["order_id", "order_item_id", "seller_id"])), how="left", on="order_id").join(sellers_df.lazy().select(pl.col(["seller_id","seller_city","seller_state"])), how="left", on="seller_id").with_columns(pl.col("order_purchase_timestamp").str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S").cast(pl.Datetime)).collect()

print(get_sellers_regions_by_order.head(20))

shape: (20, 6)
┌────────────────┬────────────────┬───────────────┬────────────────┬────────────────┬──────────────┐
│ order_id       ┆ order_purchase ┆ order_item_id ┆ seller_id      ┆ seller_city    ┆ seller_state │
│ ---            ┆ _timestamp     ┆ ---           ┆ ---            ┆ ---            ┆ ---          │
│ str            ┆ ---            ┆ i64           ┆ str            ┆ str            ┆ str          │
│                ┆ datetime[μs]   ┆               ┆                ┆                ┆              │
╞════════════════╪════════════════╪═══════════════╪════════════════╪════════════════╪══════════════╡
│ e481f51cbdc546 ┆ 2017-10-02     ┆ 1             ┆ 3504c0cb71d7fa ┆ maua           ┆ SP           │
│ 78b7cc49136f2d ┆ 00:00:00       ┆               ┆ 48d967e0e4c94d ┆                ┆              │
│ 6af7           ┆                ┆               ┆ 59d9           ┆                ┆              │
│ 53cdb2fc8bc7dc ┆ 2018-07-24     ┆ 1             ┆ 289cdb325fb7e7 ┆ belo ho

In [17]:
# Objective: Map against order date (if within the week of the holiday for that city/state/nationwide)
brazilian_holidays_transformed_df = brazilian_holidays_df.with_columns([pl.col("Holiday_Date").str.strptime(pl.Date, fmt="%m/%d/%Y").cast(pl.Datetime)]).with_columns([
    (pl.col("Holiday_Date")-pl.duration(days=3)).alias("Holiday_Week_Start"),
    (pl.col("Holiday_Date")+pl.duration(days=3)).alias("Holiday_Week_End")
])

print(brazilian_holidays_transformed_df)

shape: (120, 8)
┌──────────┬───────┬──────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ Type     ┆ State ┆ City_Lower   ┆ Holiday_Da ┆ English_Na ┆ Portuguese ┆ Holiday_We ┆ Holiday_We │
│ ---      ┆ ---   ┆ ---          ┆ te         ┆ me         ┆ _Name      ┆ ek_Start   ┆ ek_End     │
│ str      ┆ str   ┆ str          ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│          ┆       ┆              ┆ datetime[μ ┆ str        ┆ str        ┆ datetime[μ ┆ datetime[μ │
│          ┆       ┆              ┆ s]         ┆            ┆            ┆ s]         ┆ s]         │
╞══════════╪═══════╪══════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ National ┆ All   ┆ all          ┆ 2017-01-01 ┆ New Year's ┆ Ano Novo / ┆ 2016-12-29 ┆ 2017-01-04 │
│          ┆       ┆              ┆ 00:00:00   ┆ Day        ┆ Confratern ┆ 00:00:00   ┆ 00:00:00   │
│          ┆       ┆              ┆            ┆            ┆ ização     ┆ 

In [18]:
national_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="National").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(national_holidays_df)

shape: (36, 5)
┌──────────┬───────┬────────────┬─────────────────────┬─────────────────────┐
│ Type     ┆ State ┆ City_Lower ┆ Holiday_Week_Start  ┆ Holiday_Week_End    │
│ ---      ┆ ---   ┆ ---        ┆ ---                 ┆ ---                 │
│ str      ┆ str   ┆ str        ┆ datetime[μs]        ┆ datetime[μs]        │
╞══════════╪═══════╪════════════╪═════════════════════╪═════════════════════╡
│ National ┆ All   ┆ all        ┆ 2016-12-29 00:00:00 ┆ 2017-01-04 00:00:00 │
│ National ┆ All   ┆ all        ┆ 2017-04-18 00:00:00 ┆ 2017-04-24 00:00:00 │
│ National ┆ All   ┆ all        ┆ 2017-04-28 00:00:00 ┆ 2017-05-04 00:00:00 │
│ National ┆ All   ┆ all        ┆ 2017-09-04 00:00:00 ┆ 2017-09-10 00:00:00 │
│ ...      ┆ ...   ┆ ...        ┆ ...                 ┆ ...                 │
│ National ┆ All   ┆ all        ┆ 2016-06-09 00:00:00 ┆ 2016-06-15 00:00:00 │
│ National ┆ All   ┆ all        ┆ 2016-06-10 00:00:00 ┆ 2016-06-16 00:00:00 │
│ National ┆ All   ┆ all        ┆ 2016-06-21 00:0

In [19]:
#state_holidays = 
state_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="State").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(state_holidays_df)

shape: (30, 5)
┌───────┬───────┬────────────┬─────────────────────┬─────────────────────┐
│ Type  ┆ State ┆ City_Lower ┆ Holiday_Week_Start  ┆ Holiday_Week_End    │
│ ---   ┆ ---   ┆ ---        ┆ ---                 ┆ ---                 │
│ str   ┆ str   ┆ str        ┆ datetime[μs]        ┆ datetime[μs]        │
╞═══════╪═══════╪════════════╪═════════════════════╪═════════════════════╡
│ State ┆ RJ    ┆ all        ┆ 2017-04-20 00:00:00 ┆ 2017-04-26 00:00:00 │
│ State ┆ BA    ┆ all        ┆ 2017-06-29 00:00:00 ┆ 2017-07-05 00:00:00 │
│ State ┆ SE    ┆ all        ┆ 2017-07-05 00:00:00 ┆ 2017-07-11 00:00:00 │
│ State ┆ SP    ┆ all        ┆ 2017-07-06 00:00:00 ┆ 2017-07-12 00:00:00 │
│ ...   ┆ ...   ┆ ...        ┆ ...                 ┆ ...                 │
│ State ┆ RN    ┆ all        ┆ 2016-09-30 00:00:00 ┆ 2016-10-06 00:00:00 │
│ State ┆ DF    ┆ all        ┆ 2016-11-27 00:00:00 ┆ 2016-12-03 00:00:00 │
│ State ┆ PE    ┆ all        ┆ 2016-12-05 00:00:00 ┆ 2016-12-11 00:00:00 │
│ State ┆ 

In [20]:
 
city_holidays_df = brazilian_holidays_transformed_df.filter(pl.col("Type")=="City").select([
    pl.col("Type"),
    pl.col("State"),
    pl.col("City_Lower"),
    pl.col("Holiday_Week_Start"),
    pl.col("Holiday_Week_End")
])

print(city_holidays_df)

shape: (54, 5)
┌──────┬───────┬────────────────┬─────────────────────┬─────────────────────┐
│ Type ┆ State ┆ City_Lower     ┆ Holiday_Week_Start  ┆ Holiday_Week_End    │
│ ---  ┆ ---   ┆ ---            ┆ ---                 ┆ ---                 │
│ str  ┆ str   ┆ str            ┆ datetime[μs]        ┆ datetime[μs]        │
╞══════╪═══════╪════════════════╪═════════════════════╪═════════════════════╡
│ City ┆ PA    ┆ belém          ┆ 2017-01-09 00:00:00 ┆ 2017-01-15 00:00:00 │
│ City ┆ PE    ┆ rio de janeiro ┆ 2017-01-17 00:00:00 ┆ 2017-01-23 00:00:00 │
│ City ┆ PE    ┆ são vicente    ┆ 2017-01-19 00:00:00 ┆ 2017-01-25 00:00:00 │
│ City ┆ PE    ┆ são paulo      ┆ 2017-01-22 00:00:00 ┆ 2017-01-28 00:00:00 │
│ ...  ┆ ...   ┆ ...            ┆ ...                 ┆ ...                 │
│ City ┆ MG    ┆ belo horizonte ┆ 2016-12-05 00:00:00 ┆ 2016-12-11 00:00:00 │
│ City ┆ MG    ┆ dourados       ┆ 2016-12-05 00:00:00 ┆ 2016-12-11 00:00:00 │
│ City ┆ PE    ┆ santa maria    ┆ 2016-12-05 00:0

In [21]:
# get_customer_regions_by_order
# national_holidays_df
# state_holidays_df
# city_holidays_df

sql = pl.SQLContext()
sql.register("get_customer_regions_by_order",get_customer_regions_by_order.lazy())
sql.register("national_holidays_df",national_holidays_df.lazy())
sql.register("state_holidays_df",state_holidays_df.lazy())
sql.register("city_holidays_df",city_holidays_df.lazy())

In [22]:
sql.register("get_sellers_regions_by_order",get_sellers_regions_by_order.lazy())

In [23]:
isNationalHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    cross join national_holidays_df
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isNationalHolidayCustomerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""949d5b44dbf5de...",2017-11-18 00:00:00,"""f88197465ea792...","""sao goncalo do...","""RN""",2017-11-12 00:00:00,2017-11-18 00:00:00
"""403b97836b0c04...",2018-01-02 00:00:00,"""738b086814c6fc...","""rio de janeiro...","""RJ""",2017-12-29 00:00:00,2018-01-04 00:00:00
"""116f0b09343b49...",2017-12-26 00:00:00,"""3187789bec9909...","""imbituba""","""SC""",2017-12-22 00:00:00,2017-12-28 00:00:00
"""91b2a010e1e45e...",2018-05-02 00:00:00,"""cce89a605105b1...","""santo andre""","""SP""",2018-04-28 00:00:00,2018-05-04 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""148348ff65384b...","""paranavai""","""PR""",2018-06-21 00:00:00,2018-06-27 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""148348ff65384b...","""paranavai""","""PR""",2018-06-26 00:00:00,2018-07-02 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""67407057a7d5ee...","""rio de janeiro...","""RJ""",2018-06-09 00:00:00,2018-06-15 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""67407057a7d5ee...","""rio de janeiro...","""RJ""",2018-06-10 00:00:00,2018-06-16 00:00:00
"""734e7d1bbaeb2f...",2018-06-11 00:00:00,"""2932d241d1f31e...","""sao paulo""","""SP""",2018-06-09 00:00:00,2018-06-15 00:00:00


In [24]:
isNationalHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_sellers_regions_by_order
    cross join national_holidays_df
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isNationalHolidaySellerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""949d5b44dbf5de...",2017-11-18 00:00:00,"""66922902710d12...","""belo horizonte...","""MG""",2017-11-12 00:00:00,2017-11-18 00:00:00
"""403b97836b0c04...",2018-01-02 00:00:00,"""c4af86330efa7a...","""mogi das cruze...","""SP""",2017-12-29 00:00:00,2018-01-04 00:00:00
"""116f0b09343b49...",2017-12-26 00:00:00,"""ea8482cd71df3c...","""sao paulo""","""SP""",2017-12-22 00:00:00,2017-12-28 00:00:00
"""91b2a010e1e45e...",2018-05-02 00:00:00,"""7d13fca1522535...","""ribeirao preto...","""SP""",2018-04-28 00:00:00,2018-05-04 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""33cbbec1e7e104...","""bento goncalve...","""RS""",2018-06-21 00:00:00,2018-06-27 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""33cbbec1e7e104...","""bento goncalve...","""RS""",2018-06-26 00:00:00,2018-07-02 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""ef0ace09169ac0...","""sao goncalo""","""RJ""",2018-06-09 00:00:00,2018-06-15 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""ef0ace09169ac0...","""sao goncalo""","""RJ""",2018-06-10 00:00:00,2018-06-16 00:00:00
"""734e7d1bbaeb2f...",2018-06-11 00:00:00,"""d1ef48b38baca7...","""apucarana""","""PR""",2018-06-09 00:00:00,2018-06-15 00:00:00


In [25]:
isStateHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    join state_holidays_df
    on state_holidays_df.State = get_customer_regions_by_order.customer_state
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isStateHolidayCustomerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""e9ce52cd563d01...",2017-07-10 00:00:00,"""9a80c837764423...","""piracaia""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00
"""31ef15cb5a3faa...",2017-11-29 00:00:00,"""19c6be9e0d7588...","""brasilia""","""DF""",2017-11-27 00:00:00,2017-12-03 00:00:00
"""0b0f3c7a9bcb6a...",2018-07-11 00:00:00,"""7bb3b0d45b4e1a...","""sao paulo""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""32bda40452bf04...",2018-07-06 00:00:00,"""ebfb3f2e532c37...","""jaboticabal""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""418764469a9e73...",2018-07-03 00:00:00,"""b3c3c1b58dad3d...","""salvador""","""BA""",2018-06-29 00:00:00,2018-07-05 00:00:00
"""40ef667b4259d5...",2018-07-11 00:00:00,"""bb8bd97c397391...","""guarulhos""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""ed3b269b4badaf...",2018-07-12 00:00:00,"""8df348aaafe2a0...","""rancharia""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""bac5dc94cc5dbf...",2017-07-08 00:00:00,"""85fe96297110ae...","""embu-guacu""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00
"""53239db74c7835...",2017-07-11 00:00:00,"""54f60df7bf8415...","""sao paulo""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00


In [26]:
isStateHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End
    From get_sellers_regions_by_order
    join state_holidays_df
    on state_holidays_df.State = get_sellers_regions_by_order.seller_state
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isStateHolidaySellerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""a4591c265e18cb...",2017-07-09 00:00:00,"""8581055ce74af1...","""guarulhos""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00
"""c75acba615d2db...",2018-07-06 00:00:00,"""e9779976487b77...","""praia grande""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""e9ce52cd563d01...",2017-07-10 00:00:00,"""5dceca129747e9...","""santa barbara ...","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00
"""37e32b54f25d24...",2017-07-08 00:00:00,"""cc419e0650a3c5...","""santo andre""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00
"""86e574a0701d15...",2018-07-12 00:00:00,"""2b3b9ce054da76...","""sao paulo""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""0b0f3c7a9bcb6a...",2018-07-11 00:00:00,"""97512d3fc193b2...","""sao paulo""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""32bda40452bf04...",2018-07-06 00:00:00,"""ce27a3cc3c8cc1...","""sao paulo""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""40ef667b4259d5...",2018-07-11 00:00:00,"""ba143b05f0110f...","""sao paulo""","""SP""",2018-07-06 00:00:00,2018-07-12 00:00:00
"""ac244956f757ae...",2017-07-12 00:00:00,"""4e922959ae960d...","""jacarei""","""SP""",2017-07-06 00:00:00,2017-07-12 00:00:00


In [27]:
isCityHolidayCustomerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        customer_id as user_id,
        customer_city as city,
        customer_state as state,
        Holiday_Week_Start,
        Holiday_Week_End 
    From get_customer_regions_by_order
    join city_holidays_df
    on city_holidays_df.City_Lower = get_customer_regions_by_order.customer_city
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isCityHolidayCustomerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""6a0a8bfbbe7002...",2017-11-22 00:00:00,"""68451b39b13143...","""rio de janeiro...","""RJ""",2017-11-17 00:00:00,2017-11-23 00:00:00
"""bb3b61a129aba9...",2017-09-08 00:00:00,"""e986427e3d8222...","""curitiba""","""PR""",2017-09-05 00:00:00,2017-09-11 00:00:00
"""45b0a671da62d1...",2018-01-22 00:00:00,"""790cbac97e7129...","""rio de janeiro...","""RJ""",2018-01-17 00:00:00,2018-01-23 00:00:00
"""5e4c365dd3cf77...",2017-11-21 00:00:00,"""15f7712bb12fe2...","""rio de janeiro...","""RJ""",2017-11-17 00:00:00,2017-11-23 00:00:00
"""eb314576a333ad...",2018-01-23 00:00:00,"""11b404d7080235...","""rio de janeiro...","""RJ""",2018-01-17 00:00:00,2018-01-23 00:00:00
"""8713b68a2b202b...",2017-08-18 00:00:00,"""94eba5d03fc4a8...","""belo horizonte...","""MG""",2017-08-12 00:00:00,2017-08-18 00:00:00
"""2f92b812fd7c7d...",2018-01-18 00:00:00,"""18f6ca10777417...","""rio de janeiro...","""RJ""",2018-01-17 00:00:00,2018-01-23 00:00:00
"""9fa33f47b8deff...",2018-01-22 00:00:00,"""b8e0b0edff2de6...","""rio de janeiro...","""RJ""",2018-01-17 00:00:00,2018-01-23 00:00:00
"""18c2124c3874f4...",2017-11-22 00:00:00,"""3a5f82b200e600...","""rio de janeiro...","""RJ""",2017-11-17 00:00:00,2017-11-23 00:00:00


In [28]:
isCityHolidaySellerOrder = sql.query(
    """
    Select 
        order_id,
        order_purchase_timestamp,
        seller_id as user_id,
        seller_city as city,
        seller_state as state,
        Holiday_Week_Start,
        Holiday_Week_End
    From get_sellers_regions_by_order
    join city_holidays_df
    on city_holidays_df.City_Lower = get_sellers_regions_by_order.seller_city
    where order_purchase_timestamp >= Holiday_Week_Start and Holiday_Week_End >= order_purchase_timestamp
    """
)

isCityHolidaySellerOrder

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""858505c5992f35...",2018-08-16 00:00:00,"""85d9eb9ddc5d00...","""belo horizonte...","""MG""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""75d1b089800a77...",2018-08-14 00:00:00,"""85d9eb9ddc5d00...","""belo horizonte...","""MG""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""b38b3526b8b8fd...",2018-08-17 00:00:00,"""85d9eb9ddc5d00...","""belo horizonte...","""MG""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""3a84d8311ec3bc...",2018-08-12 00:00:00,"""2a84855fd20af8...","""belo horizonte...","""MG""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""6164d50a564536...",2018-08-12 00:00:00,"""289cdb325fb7e7...","""belo horizonte...","""SP""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""16ec5f5b0b5245...",2018-01-22 00:00:00,"""634964b17796e6...","""rio de janeiro...","""RJ""",2018-01-17 00:00:00,2018-01-23 00:00:00
"""4c9de71242c212...",2018-08-13 00:00:00,"""28a26ebce31d59...","""belo horizonte...","""MG""",2018-08-12 00:00:00,2018-08-18 00:00:00
"""581ef4e5af5c7b...",2017-11-18 00:00:00,"""634964b17796e6...","""rio de janeiro...","""RJ""",2017-11-17 00:00:00,2017-11-23 00:00:00
"""f5fea26ab547ee...",2017-11-20 00:00:00,"""0ad80de75c8113...","""rio de janeiro...","""RJ""",2017-11-17 00:00:00,2017-11-23 00:00:00


In [29]:
allHolidaysByOrderAll = pl.concat([isNationalHolidaySellerOrder,isStateHolidaySellerOrder,isCityHolidaySellerOrder,isNationalHolidayCustomerOrder,isStateHolidayCustomerOrder,isCityHolidayCustomerOrder])

allHolidaysByOrderAll

order_id,order_purchase_timestamp,user_id,city,state,Holiday_Week_Start,Holiday_Week_End
str,datetime[μs],str,str,str,datetime[μs],datetime[μs]
"""949d5b44dbf5de...",2017-11-18 00:00:00,"""66922902710d12...","""belo horizonte...","""MG""",2017-11-12 00:00:00,2017-11-18 00:00:00
"""403b97836b0c04...",2018-01-02 00:00:00,"""c4af86330efa7a...","""mogi das cruze...","""SP""",2017-12-29 00:00:00,2018-01-04 00:00:00
"""116f0b09343b49...",2017-12-26 00:00:00,"""ea8482cd71df3c...","""sao paulo""","""SP""",2017-12-22 00:00:00,2017-12-28 00:00:00
"""91b2a010e1e45e...",2018-05-02 00:00:00,"""7d13fca1522535...","""ribeirao preto...","""SP""",2018-04-28 00:00:00,2018-05-04 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""33cbbec1e7e104...","""bento goncalve...","""RS""",2018-06-21 00:00:00,2018-06-27 00:00:00
"""47aa4816b27ba6...",2018-06-26 00:00:00,"""33cbbec1e7e104...","""bento goncalve...","""RS""",2018-06-26 00:00:00,2018-07-02 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""ef0ace09169ac0...","""sao goncalo""","""RJ""",2018-06-09 00:00:00,2018-06-15 00:00:00
"""40c5e18f7d112b...",2018-06-11 00:00:00,"""ef0ace09169ac0...","""sao goncalo""","""RJ""",2018-06-10 00:00:00,2018-06-16 00:00:00
"""734e7d1bbaeb2f...",2018-06-11 00:00:00,"""d1ef48b38baca7...","""apucarana""","""PR""",2018-06-09 00:00:00,2018-06-15 00:00:00
